##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for beginners

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This short introduction uses [Keras](https://www.tensorflow.org/guide/keras/overview) to:

1. Build a neural network that classifies images.
2. Train this neural network.
3. And, finally, evaluate the accuracy of the model.

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

Download and install TensorFlow 2. Import TensorFlow into your program:

Note: Upgrade `pip` to install the TensorFlow 2 package. See the [install guide](https://www.tensorflow.org/install) for details.

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

# Important Links
1. About the Dataset: http://yann.lecun.com/exdb/mnist/

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [3]:
tf.__version__

'2.2.0-dev20200319'

In [4]:
# Loading the Dataset
mnist = tf.keras.datasets.mnist

In [5]:
# Seeing the dataset
mnist, type(mnist)

(<module 'tensorflow.python.keras.api._v2.keras.datasets.mnist' from '/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/api/_v2/keras/datasets/mnist/__init__.py'>,
 module)

In [6]:
# Splitting into Training and Testing set tuples
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [7]:
# Seeing the details of downloaded image data
print("x_train size is: ",x_train.shape)
print("x_test size is",x_test.shape)
print("y_train size is: ",y_train.shape)
print("y_test size is",y_test.shape)

x_train size is:  (60000, 28, 28)
x_test size is (10000, 28, 28)
y_train size is:  (60000,)
y_test size is (10000,)


Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [8]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [9]:
x_train.shape

(60000, 28, 28)

In [10]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.24332614, -0.06208158,  0.12599014,  0.09484237,  0.12149044,
        -0.4947645 ,  0.19699953,  0.1182811 ,  0.50977117, -0.05864426]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [11]:
tf.nn.softmax(predictions).numpy()

array([[0.11450418, 0.0843693 , 0.101827  , 0.0987042 , 0.10136984,
        0.05473595, 0.10932059, 0.10104503, 0.14946412, 0.08465981]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [12]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [13]:
loss_fn(y_train[:1], predictions).numpy()

2.9052346

In [14]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [15]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 1s 774us/step - loss: 0.2964 - accuracy: 0.9146
Epoch 2/5
1875/1875 [==============================] - 2s 838us/step - loss: 0.1445 - accuracy: 0.9565
Epoch 3/5
1875/1875 [==============================] - 1s 777us/step - loss: 0.1087 - accuracy: 0.9667
Epoch 4/5
1875/1875 [==============================] - 2s 805us/step - loss: 0.0888 - accuracy: 0.9733
Epoch 5/5
1875/1875 [==============================] - 1s 781us/step - loss: 0.0740 - accuracy: 0.9761


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)".

In [16]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0739 - accuracy: 0.9777


[0.07392991334199905, 0.9776999950408936]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [17]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [18]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.5105694e-08, 5.5759655e-09, 1.3814877e-05, 1.2684966e-04,
        1.8900828e-10, 2.8373330e-07, 7.9321925e-14, 9.9984217e-01,
        4.1395292e-06, 1.2836383e-05],
       [1.6375878e-07, 4.0216601e-04, 9.9955279e-01, 4.2962271e-05,
        8.1973214e-13, 1.2189679e-08, 1.2616022e-08, 6.9730110e-13,
        1.8498004e-06, 1.4587991e-12],
       [6.5331784e-07, 9.9900800e-01, 2.1275817e-04, 9.5930409e-06,
        8.8239649e-06, 2.4766059e-06, 8.4103522e-06, 2.8609380e-04,
        4.6212977e-04, 1.0175095e-06],
       [9.9920279e-01, 1.4917124e-08, 2.5383718e-04, 6.5897138e-06,
        1.1929541e-05, 1.2328878e-05, 3.6486104e-04, 1.2074256e-04,
        3.5045318e-07, 2.6533260e-05],
       [1.7975362e-06, 9.7582538e-09, 4.9340735e-05, 1.4291505e-06,
        9.8842120e-01, 2.3093703e-06, 1.9465931e-05, 6.0867862e-04,
        5.9174334e-07, 1.0895188e-02]], dtype=float32)>